# 笔记

## 概括(AI 生成+自己完善)

本教程重点介绍了在 PyTorch 中持久化模型状态的两种主要方法：保存和加载模型的**权重 (`state_dict`)** 以及保存和加载**整个模型对象**。教程以 `torchvision.models` 中的预训练模型 (VGG16) 为例，演示了如何获取并保存模型的 `state_dict`，以及如何创建一个新的模型实例并将保存的 `state_dict` 加载回去。它强调了在加载 `state_dict` 时使用 `weights_only=True` 参数以增强安全性，并在加载权重后调用 `model.eval()` 以确保推理结果的一致性。此外，教程也展示了直接保存和加载完整模型的方法，并指出了这种方法依赖于 Python 的 `pickle` 模块，需要原始类定义的可用性。最终，教程推荐使用 `state_dict` 的方式进行模型的保存与加载，认为这是更佳实践。

## 关键函数、语法(自己总结+AI 优化)

- `torchvision.models`:
    -   `torchvision` 库的一个子模块，提供了一系列常用的预训练模型架构，例如 VGG, ResNet, AlexNet 等。
    -   可以直接加载这些模型，并通过 `weights` 参数指定加载预训练的权重。例如，`weights='IMAGENET1K_V1'` 表示模型在 ImageNet-1K 数据集上预训练得到的权重。
    -   示例: `model = torchvision.models.vgg16(weights='IMAGENET1K_V1')`。
- `torch.save(object_to_save, filepath)`:
    -   (已在 `quickstart_tutorial.ipynb` 提及，此处侧重不同保存对象)
    -   **保存模型权重 (推荐方式)**: 通过 `model.state_dict()` 获取模型的状态字典 (包含所有可学习参数)，然后保存这个字典。
        -   示例: `torch.save(model.state_dict(), 'model_weights.pth')`。
    -   **保存整个模型**: 直接将模型实例传递给 `torch.save()`。这种方式会使用 Python 的 `pickle` 模块来序列化整个模型对象，包括其网络结构和代码。
        -   示例: `torch.save(model, 'model.pth')`。
- `torch.load(filepath, map_location=None, weights_only=False)`:
    -   (已在 `quickstart_tutorial.ipynb` 提及，此处侧重 `weights_only` 参数)
    -   `weights_only` (布尔类型):
        -   当设置为 `True` (推荐，尤其是在加载 `state_dict` 时): 限制反序列化过程中执行的函数，仅允许那些对于加载权重所必需的操作，有助于防止加载潜在的恶意代码。
            -   用于加载 `state_dict`: `state_dict = torch.load('model_weights.pth', weights_only=True)`，然后通过 `model.load_state_dict(state_dict)` 加载到模型实例中。
        -   当设置为 `False`: 允许执行 `pickle` 在反序列化对象时可能调用的任意代码。这在加载通过 `torch.save(model, ...)` 保存的整个模型时是必需的 (因为模型结构本身被 pickle 了)。
            -   用于加载整个模型: `loaded_model = torch.load('model.pth', weights_only=False)`。
    -   教程指出，保存和加载 `state_dict` 一般而言是更好地选择。
- `model.load_state_dict(state_dict, strict=True)`:
    -   (已在 `quickstart_tutorial.ipynb` 提及) 将从文件加载的 `state_dict` 中的参数和缓冲区加载到当前模型实例中。
    -   **注意**：在加载权重后，需要调用 `model.eval()` 将模型设置为评估模式 (特别是当模型包含 Dropout 或 BatchNorm 层时)，以获得一致的推理结果。

## 提问与解答(自己提问+AI 回答)

- **问题1 (对于保存 `state_dict` 与保存整个模型的比较):**
  教程提到 “saving `state_dict` is considered the best practice” 而不是保存整个模型。
    1.  为什么保存 `state_dict` 通常被认为是更优的方法？它与保存整个模型相比有哪些优势？
    2.  仅保存和加载 `state_dict` 有没有潜在的缺点或限制？

  - **解答:**
    1.  **保存 `state_dict` 的优势:**
        -   **可移植性 (Portability)**:
            -   `state_dict` 是一个简单的 Python 字典，它只存储模型的参数和持久化缓冲区（如 BatchNorm 的运行统计量）的张量数据，而不包含模型的代码结构。这使得它在不同的项目、不同的 PyTorch 版本之间，甚至在模型代码有少量重构（只要参数名称和形状保持一致）时，通常更容易迁移和加载。
            -   保存整个模型会使用 Python 的 `pickle` 模块序列化整个对象，包括其类定义。如果加载模型的环境没有完全相同的类定义（例如，类名更改、文件路径更改、或者某些依赖的自定义模块不可用），加载可能会失败或出现问题。
        -   **安全性 (Security)**:
            -   当使用 `torch.load(filepath, weights_only=True)` 加载 `state_dict` 时，PyTorch 会限制反序列化过程中可执行的代码，这极大地降低了从不受信任的来源加载模型文件时执行恶意代码的风险。
            -   加载通过 `torch.save(model, ...)` 保存的整个模型时，`pickle` 模块理论上可以执行任意代码，如果模型文件被篡改，可能存在安全隐患。因此，加载整个模型时需要确保文件来源可靠，并且通常需要设置 `weights_only=False`。
        -   **灵活性 (Flexibility)**:
            -   `state_dict` 使得在模型架构有细微变化时加载权重成为可能（例如，在迁移学习中，可能只加载预训练模型的部分层）。可以先修改 `state_dict` 字典（如过滤掉不匹配的键），然后再加载到新的模型结构中。
            -   加载整个模型则要求模型结构严格一致。

    2.  **仅保存和加载 `state_dict` 的缺点或限制:**
        -   **需要模型类定义**: 加载 `state_dict` 之前，必须先创建一个与保存 `state_dict` 时结构相匹配（至少是参数名和形状要兼容）的模型实例。如果模型的 Python 类定义不可用，就无法实例化模型，也就无法加载 `state_dict`。
        -   **不保存优化器状态或训练配置**: `state_dict` 通常只包含模型参数。如果需要从某个训练断点恢复训练，还需要单独保存和加载优化器的状态 (`optimizer.state_dict()`) 以及当前的 epoch、损失等其他训练元数据。这通常通过创建一个包含所有这些信息的“检查点 (checkpoint)”字典来完成。
        -   **不保存完整的代码逻辑**: 如果模型类中除了标准的 `nn.Module` 层之外还有一些自定义的辅助方法或属性（非参数、非缓冲区），这些不会被 `state_dict` 保存。

    参考资料:
    -   PyTorch官方文档 - Serialization Semantics: [https://pytorch.org/docs/stable/notes/serialization.html](https://pytorch.org/docs/stable/notes/serialization.html) (详细讨论了保存和加载的不同方法及其优缺点)

- **问题2 (关于加载预训练模型 `models.vgg16(weights='IMAGENET1K_V1')`):**
  教程使用 `torchvision.models` 来加载预训练的 VGG16 模型。在实际应用中，我们经常需要对这些预训练模型进行修改以适应新的任务（例如，改变最后分类层的输出类别数），这被称为迁移学习。
    1.  如果我们加载了预训练的 VGG16，然后修改了其 `classifier` 部分（例如，`model.classifier[6] = nn.Linear(num_ftrs, new_num_classes)`），之后我们尝试加载之前保存的原始 VGG16 的 `state_dict`，`model.load_state_dict()` 的默认行为 (`strict=True`) 会怎样？
    2.  在迁移学习中，当模型结构（尤其是分类头）发生改变后，有哪些常见的策略可以只加载 `state_dict` 中与新模型结构匹配的部分权重（例如，只加载特征提取部分的权重）？

  - **解答:**
    1.  **`load_state_dict(strict=True)` 在结构修改后的行为:**
        -   如果加载的 `state_dict` 包含的键（参数名）与当前模型实例 (`model`) 的 `state_dict` 中的键不完全匹配，或者对应参数的形状不匹配，那么 `model.load_state_dict(saved_state_dict, strict=True)` (默认行为) 将会抛出一个 `RuntimeError`。
        -   具体到修改 VGG16 的 `classifier[6]`：
            -   原始 `state_dict` 中会有键如 `classifier.6.weight` 和 `classifier.6.bias`，它们的形状对应原始的1000个输出类别。
            -   修改后的模型 `classifier.6` 是一个新的 `nn.Linear` 层，其权重和偏置的形状将对应 `new_num_classes`。
            -   因此，由于 `classifier.6.weight` 和 `classifier.6.bias` 的形状不匹配，`strict=True` 的加载会失败。

    2.  **迁移学习中加载部分权重的策略:**
        当模型结构（特别是分类头）发生改变，而我们只想加载预训练模型中匹配的部分（通常是特征提取层）时，有几种常见策略：
        -   **过滤 `state_dict`**:
            -   加载预训练模型的 `state_dict` 到一个临时的字典中。
            -   创建一个新的字典，只包含那些键名存在于当前新模型 `model.state_dict()` 中，并且形状也匹配的参数。
            -   然后使用 `model.load_state_dict(filtered_pretrained_state_dict, strict=False)` 或 `model.load_state_dict(filtered_pretrained_state_dict, strict=True)` (如果能确保过滤后完全匹配) 来加载。
            ```python
            # pretrained_dict = torch.load('original_vgg16_weights.pth')
            # model_dict = model.state_dict() # model 是修改后的新模型
            #
            # # 1. Filter out unnecessary keys
            # filtered_pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict and model_dict[k].shape == v.shape}
            # # 2. Overwrite entries in the existing state dict
            # model_dict.update(filtered_pretrained_dict)
            # # 3. Load the new state dict
            # model.load_state_dict(model_dict)
            ```
        -   **使用 `strict=False`**:
            -   直接调用 `model.load_state_dict(pretrained_state_dict, strict=False)`。这会加载 `pretrained_state_dict` 中所有与当前模型键名和形状都匹配的参数，并忽略那些不匹配的键（例如新的分类头或者被移除的层）以及形状不匹配的键。
            -   优点是简单，但缺点是如果意外地有不匹配的键，它不会报错，可能导致某些期望加载的权重被静默地跳过。因此需要仔细检查哪些权重被加载了。
        -   **逐层或逐参数加载**:
            -   对于更复杂的场景，可以迭代预训练的 `state_dict`，并根据参数名称和形状手动将其值赋给新模型的对应参数。这种方式控制最精细，但也最繁琐。

        在迁移学习中，通常的做法是加载预训练模型除了最后分类层之外的所有层的权重，然后随机初始化新的分类层，并主要训练这个新的分类层（或者在后期微调整个网络）。

    参考资料:
    -   PyTorch 教程 - Transfer Learning for Computer Vision: [https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html](https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html) (该教程展示了如何修改模型并只训练新层)
    -   PyTorch `torch.nn.Module.load_state_dict` 文档: [https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module.load_state_dict](https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module.load_state_dict)


[Learn the Basics](intro.html) \|\|
[Quickstart](quickstart_tutorial.html) \|\|
[Tensors](tensorqs_tutorial.html) \|\| [Datasets &
DataLoaders](data_tutorial.html) \|\|
[Transforms](transforms_tutorial.html) \|\| [Build
Model](buildmodel_tutorial.html) \|\|
[Autograd](autogradqs_tutorial.html) \|\|
[Optimization](optimization_tutorial.html) \|\| **Save & Load Model**

Save and Load the Model
=======================

In this section we will look at how to persist model state with saving,
loading and running model predictions.


In [1]:
import torch
import torchvision.models as models

Saving and Loading Model Weights
================================

PyTorch models store the learned parameters in an internal state
dictionary, called `state_dict`. These can be persisted via the
`torch.save` method:


In [2]:
model = models.vgg16(weights='IMAGENET1K_V1')
torch.save(model.state_dict(), 'model_weights.pth')

To load model weights, you need to create an instance of the same model
first, and then load the parameters using `load_state_dict()` method.

In the code below, we set `weights_only=True` to limit the functions
executed during unpickling to only those necessary for loading weights.
Using `weights_only=True` is considered a best practice when loading
weights.


In [3]:
model = models.vgg16() # we do not specify ``weights``, i.e. create untrained model
model.load_state_dict(torch.load('model_weights.pth', weights_only=True))
model.eval()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

<div style="background-color: #54c7ec; color: #fff; font-weight: 700; padding-left: 10px; padding-top: 5px; padding-bottom: 5px"><strong>NOTE:</strong></div>

<div style="background-color: #f3f4f7; padding-left: 10px; padding-top: 10px; padding-bottom: 10px; padding-right: 10px">

<p>be sure to call <code>model.eval()</code> method before inferencing to set the dropout and batch normalization layers to evaluation mode. Failing to do this will yield inconsistent inference results.</p>

</div>



Saving and Loading Models with Shapes
=====================================

When loading model weights, we needed to instantiate the model class
first, because the class defines the structure of a network. We might
want to save the structure of this class together with the model, in
which case we can pass `model` (and not `model.state_dict()`) to the
saving function:


In [4]:
torch.save(model, 'model.pth')

We can then load the model as demonstrated below.

As described in [Saving and loading
torch.nn.Modules](https://pytorch.org/docs/main/notes/serialization.html#saving-and-loading-torch-nn-modules),
saving `state_dict` is considered the best practice. However, below we
use `weights_only=False` because this involves loading the model, which
is a legacy use case for `torch.save`.


In [5]:
model = torch.load('model.pth', weights_only=False),

<div style="background-color: #54c7ec; color: #fff; font-weight: 700; padding-left: 10px; padding-top: 5px; padding-bottom: 5px"><strong>NOTE:</strong></div>

<div style="background-color: #f3f4f7; padding-left: 10px; padding-top: 10px; padding-bottom: 10px; padding-right: 10px">

<p>This approach uses Python <a href="https://docs.python.org/3/library/pickle.html">pickle</a> module when serializing the model, thus it relies on the actual class definition to be available when loading the model.</p>

</div>



Related Tutorials
=================

-   [Saving and Loading a General Checkpoint in
    PyTorch](https://pytorch.org/tutorials/recipes/recipes/saving_and_loading_a_general_checkpoint.html)
-   [Tips for loading an nn.Module from a
    checkpoint](https://pytorch.org/tutorials/recipes/recipes/module_load_state_dict_tips.html?highlight=loading%20nn%20module%20from%20checkpoint)
